<a href="https://colab.research.google.com/github/Danc2050/Distributed-Computing-Scripts/blob/master/google-colab/GoogleColabGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <strong> <h1>📔 Colab GPU and CPU Notebook</strong></h1> 
#{ output-height: 8, form-width: "100%", display-mode: "form" }

#@markdown <h4>↖️ Click the ▶ button after deciding on the options below.</h4>
#@markdown <h4>🔌 Make sure the GPU is enabled under <em>Runtime→Change runtime type</em></h4>
#@markdown <h4>💡 Keep each notebook <strong>open</strong> in your browser to prevent disconnection. (📌 pin the tab or 🗗 move it to a dedicated window)</h4>
#@markdown <h4>🕷️ A <em>debug</em> field outputs CUDALucas or Prime95 progress + GPU specs, then exits.</h4>
#@markdown <h4>📜 See the <a href="https://github.com/Danc2050/Distributed-Computing-Scripts/tree/master/google-colab" target="_blank">source code</a> for more information.</h4>
#@markdown <h4>🤷‍♂️ Optionally, create a GIMPS/PrimeNet account <a href="https://www.mersenne.org/update/" target="_blank">here</a>.</h4>

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
prime_password = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'test2' #@param ['Default'] {allow-input: true}
GPU_type_of_work = '100 - First time LL tests' #@param ['100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests']
CPU_type_of_work = '100 - First time LL tests' #@param ['0 - Whatever makes the most sense', '100 - First time LL tests', '101 - Double-check LL tests', '102 - World record LL tests', '104 - 100 million digit LL tests', '150 - First time PRP tests', '151 - Double-check PRP tests', '152 - World record PRP tests', '154 - 100 million digit PRP tests']
computer_number = 1 #@param[1, 2]
output_type = 'GPU (CUDALucas)' #@param ['GPU (CUDALucas)', 'CPU (Prime95)'] 
debug = 'False' #@param ['False', 'GPU (CUDALucas)', 'CPU (Prime95)']
gpu_info = ""


#from google.colab import drive
import os

def run():
  # Run CUDALucas and Mprime
  ini = ".ini"
  out = ".out"
  txt = ".txt"
  #!cd cudalucas; nohup python2 primenet.py -d -T $GPU_type_of_work -u $prime_ID -p $prime_password &
  !cd cudalucas; nohup python2 primenet.py -d -T $GPU_type_of_work -u $prime_ID -p $prime_password -i "worktodo$computer_number$txt"
  !cd cudalucas; ls
  !sleep 1.5
  #./mprime -A$computer_number -d
  #./CUDALucas -i "CUDALucas$computer_number.ini"
  if output_type == 'GPU (CUDALucas)':
    !cd mprime_gpu && chmod 777 mprime; nohup ./mprime -A$computer_number -d >> "cpu$computer_number$out" &
    print("HERE")
    !cd cudalucas && chmod 777 CUDALucas && ./CUDALucas -i "CUDALucas$computer_number$ini" | tee -a "gpu$computer_number$out"
    #!cd mprime_gpu && chmod 777 mprime; nohup ./mprime -d >> cpu$computer_number.out &
    #!cd cudalucas && chmod 777 CUDALucas && ./CUDALucas | tee -a gpu$computer_number.out
  else:
    !cd cudalucas && chmod 777 CUDALucas; nohup ./CUDALucas -i "CUDALucas$computer_number$ini" >> "gpu$computer_number$out" &
    !cd mprime_gpu && chmod 777 mprime; && ./mprime -A$computer_number -d | tee -a "cpu$computer_number$out"
    #!cd cudalucas && chmod 777 CUDALucas; nohup ./CUDALucas >> gpu$computer_number.out &
    #!cd mprime_gpu && chmod 777 mprime && ./mprime -d | tee -a cpu$computer_number.out

def install():
  # Download/Install/Configure CUDALucas
  !wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
  print(gpu_info) # Output what GPU is assigned to this Notebook
  #!wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/cudalucas.sh -q # download CUDALucas script
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/cudalucas2.sh -q # download CUDALucas script
  !sed -i '/^if ! COMPUTE=/,/^fi/!b; /^fi/a echo "$COMPUTE"' cudalucas2.sh # Output CUDA compute capability of GPU
  !sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_35,code=sm_35 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' cudalucas2.sh
  !sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas2.sh # Do not check for an Nvidia GPU
  !sed -i '/^nohup / s/^/# /' cudalucas2.sh # Do not start PrimeNet
  !sed -i '/^crontab / s/^/# /' cudalucas2.sh # Do not create a cronjob
  !bash -- cudalucas2.sh $computer_number "" $prime_ID $GPU_type_of_work # Do not pass any additional parameters here

  # Download/Install/Configure Prime95
  !rm mprime.sh # remove any old download of the Prime95 script
  !wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/mprime2.sh -q # Download Prime95 script
  !sed -i '/^nohup / s/^/# /' mprime2.sh # Do not start Prime95
  !sed -i '/^crontab / s/^/# /' mprime2.sh # Do not create a cronjob
  !sed -i 's/"mprime"/"mprime_gpu"/' mprime2.sh # Name the folder specific to the runtime type
  !bash -- mprime2.sh $computer_number $prime_ID $computer_name $CPU_type_of_work # Run script
  run()

def debug_exit():
  !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this notebook
  print(debug == 'Prime95') 
  print(os.path.exists('mprime_gpu/nohup.out'))
  if debug == 'GPU (CUDALucas)' and os.path.exists('cudalucas/nohup.out'):
    !tail -n 100 cudalucas/nohup.out # view CUDALucas progress
  elif debug == 'CPU (Prime95)' and os.path.exists('mprime_gpu/nohup.out'):
    !tail -n 100 mprime_gpu/nohup.out # view CUDALucas progress
  else:
    print(f'No {debug}/nohup.out file found for debug option.')

# Load & cd into gdrive for persistent data
#drive.mount('/content/gdrive')
#%cd gdrive/'My Drive'
%cd drive/'My Drive'

# use/cleanup input from user
prime_ID = 'psu' if prime_ID.lower() == 'default' else prime_ID
computer_name = "" if computer_name.lower() == 'default' else computer_name
prime_password = 'vikings' if prime_password.lower() == 'default' else prime_password
CPU_type_of_work = CPU_type_of_work.split("-")[0].rstrip()
GPU_type_of_work = GPU_type_of_work.split("-")[0].rstrip()
debug = False if debug == 'False' else debug

# Add quotes to string args so script can parse spaces/special characters
prime_ID = f'"'+ prime_ID + f'"'
computer_name = f'"'+ computer_name + f'"'
prime_password = f'"'+ prime_password + f'"'

# GPU Check
gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
gpu_info = "\n".join(gpu_info)

#if True:
#  !echo $computer_number
#  !echo "CUDALucascomputer_number$ini"

if gpu_info.find('failed') >= 0:
  print('Select the "Runtime" → "Change runtime type" → "GPU" → "SAVE" to enable a GPU accelerator, ')
  print('and then re-execute this cell.')

elif debug:
  debug_exit()

elif os.path.exists('mprime_gpu') and os.path.exists('cudalucas') :
  run()

else:
  install()

print("Gracefully exiting...")